In [2]:
import pandas as pd
import numpy as np
from pypfopt import risk_models, expected_returns, plotting, EfficientFrontier
import matplotlib.pyplot as plt
from datetime import date
#pypfopt.__version__
import quantstats as qs
import yfinance as yf
import tempfile
import plotly.express as px

https://medium.com/enjoy-algorithm/pre-processing-of-time-series-data-c50f8a3e7a98

https://www.kaggle.com/code/harshalnikose/portfolio-management
https://medium.com/cloudcraftz/portfolio-trading-using-pyportfolioopt-24040247249


In [6]:
def test_ef():
    companies = ["MSFT", "AMZN", "META", "BABA", "GE", "GOOG", "AMD", "WMT", "BAC", "GM", "T", "UAA", "MA", "PFE", "JPM", "SBUX"]
    stock_data = yf.download(companies, start="2020-6-01", end=date.today())
    stock_data = stock_data["Adj Close"]
    mu2 = expected_returns.mean_historical_return(stock_data)
    cov_matrix2 = risk_models.sample_cov(stock_data)
    temp_ef = EfficientFrontier(mu2, cov_matrix2)
    return temp_ef

temp = qs.utils.download_returns(companies)

In [3]:
companies = ["MSFT", "AMZN", "META", "BABA", "GE", "GOOG", "AMD", "WMT", "BAC", "GM", "T", "UAA", "MA", "PFE", "JPM", "SBUX"]

In [ ]:
stock_data = yf.download(companies, start="2020-6-01", end=date.today())

In [ ]:
stock_data = stock_data["Adj Close"]

In [ ]:
stock_data

In [ ]:
mu2 = expected_returns.mean_historical_return(stock_data)
print(mu2)

In [ ]:
stock_data.pct_change()[1:].cumsum().plot()

In [ ]:
cov_matrix2 = risk_models.sample_cov(stock_data)
print(cov_matrix2)
type(cov_matrix2)

In [ ]:
plotting.plot_covariance(cov_matrix2, plot_correlation=True)

In [ ]:
ef = EfficientFrontier(mu2, cov_matrix2)
ef.max_sharpe()

In [ ]:
weights = ef.clean_weights()
pd.Series(weights).plot.pie()

In [ ]:
def plot_weights(ef):
    ef.max_sharpe()
    weights = ef.clean_weights()
    df = pd.DataFrame(weights,index=['Weight'])
    df = df.T
    df = df[df['Weight']!=0] # Remove 0 values
    fig = px.pie(df,values='Weight',names=df.index)
    return fig

In [ ]:
plot_weights(ef2)

In [9]:
ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 25.4%
Annual volatility: 22.1%
Sharpe Ratio: 1.06


(0.2535719095254672, 0.2205031405726748, 1.0592679492856707)

In [ ]:
ef = EfficientFrontier(mu2, cov_matrix2, weight_bounds=(None, None))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.show()

In [ ]:
ef = EfficientFrontier(mu2, cov_matrix2, weight_bounds=(None, None))
fig, ax = plt.subplots()
ef_max_sharpe = ef.deepcopy()


# Find the tangency portfolio
ef_max_sharpe.max_sharpe()
ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Min Volatility
ef_min_vol = ef.deepcopy()
ef_min_vol.min_volatility()
vol_ret, vol_std, _ = ef_min_vol.portfolio_performance()
ax.scatter(vol_std, vol_ret, marker="*", s=100, c="g", label="Min Volatility")

plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)

# Generate random portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
rets = w.dot(ef.expected_returns)
stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
ax.set_title("Efficient Frontier with random portfolios")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_ef_line(
    opt,
    ef_param="return",
    ef_param_range=None,
    points=100,
    ax=None,
    show_assets=True,
    show_tickers=False,
    **kwargs
):

    ax = ax or plt.gca()

    if isinstance(opt, CLA):
        ax = _plot_cla(
            opt, points, ax=ax, show_assets=show_assets, show_tickers=show_tickers
        )
    elif isinstance(opt, EfficientFrontier):
        if ef_param_range is None:
            ef_param_range = _ef_default_returns_range(opt, points)

        ax = _plot_ef(
            opt,
            ef_param,
            ef_param_range,
            ax=ax,
            show_assets=show_assets,
            show_tickers=show_tickers,
        )
    else:
        raise NotImplementedError("Please pass EfficientFrontier or CLA object")

    ax.legend()
    ax.set_xlabel("Volatility")
    ax.set_ylabel("Return")

    _plot_io(**kwargs)
    return ax


In [ ]:
def _plot_ef(ef, ef_param='return', ef_param_range, ax, show_tickers):
    """
    Helper function to plot the efficient frontier from an EfficientFrontier object
    """
    mus, sigmas = [], []

    # Create a portfolio for each value of ef_param_range
    for param_value in ef_param_range:
        try:
            if ef_param == "utility":
                ef.max_quadratic_utility(param_value)
            elif ef_param == "risk":
                ef.efficient_risk(param_value)
            elif ef_param == "return":
                ef.efficient_return(param_value)
            else:
                raise NotImplementedError(
                    "ef_param should be one of {'utility', 'risk', 'return'}"
                )
        except exceptions.OptimizationError:
            continue
        except ValueError:
            warnings.warn(
                "Could not construct portfolio for parameter value {:.3f}".format(
                    param_value
                )
            )

        ret, sigma, _ = ef.portfolio_performance()
        mus.append(ret)
        sigmas.append(sigma)

    ax.plot(sigmas, mus, label="Efficient frontier")

    asset_mu = ef.expected_returns
    asset_sigma = np.sqrt(np.diag(ef.cov_matrix))
    
    return ax

In [ ]:
ef = EfficientFrontier(mu2, cov_matrix2, weight_bounds=(None, None))


def plot_ef_with_random(ef, n_samples=10000):
    fig, ax = plt.subplots()
    ef_max_sharpe = ef.deepcopy()
    ef_min_vol = ef.deepcopy()

    # Find the tangency portfolio
    ef_max_sharpe.max_sharpe()
    ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
    ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

    # Min Volatility
    ef_min_vol.min_volatility()
    vol_ret, vol_std, _ = ef_min_vol.portfolio_performance()
    ax.scatter(vol_std, vol_ret, marker="*", s=100, c="g", label="Min Volatility")


    # Generate random portfolios
    w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
    rets = w.dot(ef.expected_returns)
    stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
    sharpes = rets / stds
    ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

    # Output
    plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)
    ax.set_title("Efficient Frontier with random portfolios")
    ax.legend()
    plt.tight_layout()
    
    st.pyplot(fig)

In [2]:
def plot_ef_with_random(ef, output=None, n_samples=10000):
    fig, ax = plt.subplots()
    ef_max_sharpe = ef.deepcopy()
    ef_min_vol = ef.deepcopy()

    # Find the tangency portfolio
    ef_max_sharpe.max_sharpe()
    ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
    ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

    # Min Volatility
    ef_min_vol.min_volatility()
    vol_ret, vol_std, _ = ef_min_vol.portfolio_performance()
    ax.scatter(vol_std, vol_ret, marker="*", s=100, c="g", label="Min Volatility")


    # Generate random portfolios
    w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
    rets = w.dot(ef.expected_returns)
    stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
    sharpes = rets / stds
    ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

    # Output
    plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)
    ax.set_title("Efficient Frontier with random portfolios")
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_min_volatility(ef):
    ef_min_vol = ef.deepcopy()
    # Min Volatility
    ef_min_vol.min_volatility()
    vol_ret, vol_std, _ = ef_min_vol.portfolio_performance()
    ax.scatter(vol_std, vol_ret, marker="*", s=100, c="g", label="Min Volatility")
    
def plot_max_sharpe(ef):
    ef_max_sharpe = ef.deepcopy()
    # Find the tangency portfolio
    ef_max_sharpe.max_sharpe()
    ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
    ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

In [ ]:
def plot_ef_with_random(ef, output=None, n_samples=10000):
    fig, ax = plt.subplots()
    plot_min_volatility(ef)
    plot_max_sharpe(ef)


    # Generate random portfolios
    w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
    rets = w.dot(ef.expected_returns)
    stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
    sharpes = rets / stds
    ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

    # Output
    plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)
    ax.set_title("Efficient Frontier with random portfolios")
    ax.legend()
    plt.tight_layout()
    plt.show()

In [4]:
def plot_weights(ef):
    ef2 = ef.deepcopy()
    ef2.max_sharpe()
    weights = ef2.clean_weights()
    df = pd.DataFrame(weights,index=['Weight'])
    df = df.T
    df = df[df['Weight']!=0] # Remove 0 values
    fig = px.pie(df,values='Weight',names=df.index)
    return fig

In [7]:
ef_o = test_ef()
ef = ef_o.deepcopy()

[*********************100%%**********************]  16 of 16 completed


In [11]:
import plotly.graph_objects as go
import warnings
from pypfopt import EfficientFrontier, exceptions

def plot_ef(ef,  ef_param_range):
    """
    Helper function to plot the efficient frontier from an EfficientFrontier object
    """
    mus, sigmas = [], []

    # Create a portfolio for each value of ef_param_range
    for param_value in ef_param_range:
        try:
            ef.efficient_return(param_value)
        except exceptions.OptimizationError:
            continue
        except ValueError:
            warnings.warn(
                "Could not construct portfolio for parameter value {:.3f}".format(
                    param_value
                )
            )

        ret, sigma, _ = ef.portfolio_performance()
        mus.append(ret)
        sigmas.append(sigma)

    return go.Scatter(x=sigmas, y=mus, mode='lines', name='Efficient Frontier')

OrderedDict([('AMD', 0.14639),
             ('AMZN', -0.53113),
             ('BABA', -0.26261),
             ('BAC', -0.72542),
             ('GE', 1.0),
             ('GM', -0.27946),
             ('GOOG', 0.45403),
             ('JPM', 1.0),
             ('MA', -0.40558),
             ('META', -0.08622),
             ('MSFT', 0.95007),
             ('PFE', -0.21078),
             ('SBUX', 0.19913),
             ('T', -0.32868),
             ('UAA', -0.36941),
             ('WMT', 0.44967)])